In [90]:
import pandas as pd
import numpy as np
import json
import re
import spacy
from nltk import ngrams, everygrams

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner', 'tagger'])

from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC, SVC

from sklearn.metrics import accuracy_score

### Data

In [2]:
train = pd.read_json('train.json').set_index('id')

In [3]:
train

,cuisine,ingredients
id,,
10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
22213,indian,"[water, vegetable oil, wheat, salt]"
13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."
...,...,...
29109,irish,"[light brown sugar, granulated sugar, butter, ..."
11462,italian,"[KRAFT Zesty Italian Dressing, purple onion, b..."
2238,irish,"[eggs, citrus fruit, raisins, sourdough starte..."


In [4]:
test = pd.read_json('test.json').set_index('id')

In [5]:
test

,ingredients
id,
18009,"[baking powder, eggs, all-purpose flour, raisi..."
28583,"[sugar, egg yolks, corn starch, cream of tarta..."
41580,"[sausage links, fennel bulb, fronds, olive oil..."
29752,"[meat cuts, file powder, smoked sausage, okra,..."
35687,"[ground black pepper, salt, sausage casings, l..."
...,...
30246,"[large egg yolks, fresh lemon juice, sugar, bo..."
36028,"[hot sauce, butter, sweet potatoes, adobo sauc..."
22339,"[black pepper, salt, parmigiano reggiano chees..."


In [7]:
with open('sample_submission.csv') as f:
    sample_submission = f.readlines()

In [8]:
len(sample_submission)

9945

In [9]:
sample_submission[:5]

['id,cuisine\n',
 '35203,italian\n',
 '17600,italian\n',
 '35200,italian\n',
 '17602,italian\n']

### Feature engineering

In [6]:
def clean_ingredients(old_ing):
    ing = old_ing.lower()
    ing = re.sub(r'\(.*oz\.\)', ' ', ing)
    ing = re.sub(r'[^a-zâçèéíîú]', ' ', ing)
    ing = re.sub(r' +', ' ', ing)
    ing = ing.strip()
    return ing

In [7]:
train['ingredients'] = train['ingredients'].apply(lambda x: [clean_ingredients(ing) for ing in x])
test['ingredients'] = test['ingredients'].apply(lambda x: [clean_ingredients(ing) for ing in x])

In [66]:
def lemmatize_2(old_ing):
    words = ' '.join(old_ing.split()[-2:])
    doc = nlp(words)
    return ' '.join([token.lemma_ for token in doc])

In [67]:
train['norm'] = train['ingredients'].apply(lambda x: [lemmatize_2(ing) for ing in x])
test['norm'] = test['ingredients'].apply(lambda x: [lemmatize_2(ing) for ing in x])

In [128]:
def get_ngrams(l: list, n: int):
    s = ' '.join(l)
    ngrams_list = []
    for item in s.split():
        grams = list(everygrams(item, max_len=n, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='<e>'))
        ngrams_list += [''.join(let) for let in grams]
    return list(set(ngrams_list))

In [152]:
train['ngrams'] = train['ingredients'].apply(lambda x: get_ngrams(x, 3))
test['ngrams'] = test['ingredients'].apply(lambda x: get_ngrams(x, 3))

In [153]:
ing_dict_train = train[['ngrams']].to_dict(orient='records')
ing_dict_test = test[['ngrams']].to_dict(orient='records')

In [154]:
vectorizer = DictVectorizer()

In [155]:
X = vectorizer.fit_transform(ing_dict_train)
X_kaggle = vectorizer.transform(ing_dict_test)

In [156]:
print(X.shape)
print(X_kaggle.shape)

(39774, 4059)
(9944, 4059)


In [157]:
y = train['cuisine']
y

id
10259          greek
25693    southern_us
20130       filipino
22213         indian
13162         indian
            ...     
29109          irish
11462        italian
2238           irish
41882        chinese
2362         mexican
Name: cuisine, Length: 39774, dtype: object

### LinearSVC

In [158]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=8000, random_state=10)

In [ ]:
model = LinearSVC(random_state=10)

In [18]:
%%time
model.fit(X_train, y_train)

Wall time: 4.6 s


LinearSVC()

In [19]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [20]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9320513627494178
0.771125


In [21]:
cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

0.7674109852582582

Cross-validation score: 0.767

### SVC

Cross-val score kernel='poly': 0.604<br>
Cross-val score kernel='rbf': 0.757<br>
Cross-val score kernel='linear': 0.756<br>
Cross-val score kernel='sigmoid': 0.744

In [44]:
model = SVC(random_state=10)

In [45]:
%%time
model.fit(X_train, y_train)

Wall time: 1min 30s


SVC(random_state=10)

In [46]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [47]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.9012400075533455
0.76175


In [48]:
cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

0.7572284444480969

### Hyperparam tuning for LinearSVC

In [80]:
def get_best_model(model, X_tr, y_tr, param_grid):
    
    grid = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')
    grid.fit(X_tr, y_tr)
    
    return grid.best_estimator_, grid.best_score_, grid.best_params_, pd.DataFrame(grid.cv_results_)

In [81]:
model = LinearSVC(random_state=10, max_iter=10000)

In [82]:
params = {'penalty': ['l1', 'l2'],
          'class_weight': [None, 'balanced'],
          'C': [1e-7, 1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1, 1e+1, 1e+2, 1e+3]}

In [83]:
model_data = get_best_model(model, X, y, params)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number 

In [84]:
for item in model_data[:3]:
    print(item)

LinearSVC(C=0.1, max_iter=10000, random_state=10)
0.7804595967214764
{'C': 0.1, 'class_weight': None, 'penalty': 'l2'}


Best LinearSVC for normalized last 2 words:<br>
LinearSVC(C=0.1, max_iter=10000, random_state=10)<br>
0.7804595967214764<br>
{'C': 0.1, 'class_weight': None, 'penalty': 'l2'}

### LinearSVC with best params

In [159]:
model = LinearSVC(C=0.1, max_iter=10000, random_state=10)

In [160]:
model.fit(X_train, y_train)

LinearSVC(C=0.1, max_iter=10000, random_state=10)

In [161]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [162]:
print(accuracy_score(y_train, pred_train))
print(accuracy_score(y_test, pred_test))

0.886983067917165
0.774875


In [163]:
cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

0.7799316212786691

Cross-validation score: 0.78

### Kaggle submission

In [85]:
model = LinearSVC(C=0.1, max_iter=10000, random_state=10)

In [86]:
model.fit(X, y)

LinearSVC(C=0.1, max_iter=10000, random_state=10)

In [87]:
pred_kaggle = model.predict(X_kaggle)
pred_kaggle

array(['irish', 'southern_us', 'italian', ..., 'italian', 'southern_us',
       'mexican'], dtype=object)

In [88]:
submission = pd.DataFrame(pred_kaggle, index=test.index, columns=['cuisine'])
submission

,cuisine
id,
18009,irish
28583,southern_us
41580,italian
29752,cajun_creole
35687,italian
...,...
30246,french
36028,southern_us
22339,italian


In [89]:
submission.to_csv('submission5.csv')

For normalized last 2 words:<br>
Cross-val score: 0.78408<br>
Kaggle score: 0.78710

For char ngrams:<br>
Cross-val score: 0.78